In [1]:
import  numpy as np
import tensorflow as tf
import numpy as np
from collections import Counter
import random
import torch.nn.functional
import math
def load_data(data_path):
    data =[]
    ids =[]
    max_len =0
    law_labels =[]
    money_label =[]
    with open(data_path,'r',encoding='utf8') as f:
        for line in f:
            line = line.strip()
            lines = line.split('\t')
            ids.append(lines[0])

            sentence = lines[1].split(' ')
            tmp_len = len(sentence)
            if(tmp_len>max_len):
                max_len = tmp_len
            multi_label =[int(i)-1 for i in lines[3].split(',')]
            data.append(sentence)
            law_labels.append(multi_label)
            money_label.append(lines[2])
    return ids , data, money_label,law_labels
data_path ='./data/AI_law/sample_seg_train.txt'
ids,data,money_label,law_labels = load_data(data_path)


In [20]:
from itertools import groupby
def text_len_distrubution(data):
    len_list = [len(text) for text in data]
    #print(len_list[:20])
    step =1000
    for k, v in groupby(sorted(len_list),key= lambda x:(x-1)//step):
        print('{}-{}'.format(k*step+1, (k+1)*step)+":"+str(len(list(v))))
text_len_distrubution(data)

1-1000:8421
1001-2000:950
2001-3000:331
3001-4000:125
4001-5000:70
5001-6000:36
6001-7000:16
7001-8000:11
8001-9000:11
9001-10000:12
10001-11000:2
11001-12000:1
12001-13000:5
14001-15000:3
15001-16000:1
16001-17000:1
17001-18000:3
20001-21000:1


In [3]:
def bulid_vocabulary(data,min_count =3):
    count = [('<UNK>', -1), ('<PAD>', -1)]
    words =[]
    [words.extend(line) for line in data]
    counter = Counter(words)
    counter_list = counter.most_common()
    for word,c in counter_list:
        #记录最少出现三次的词
        if c>min_count:
            count.append((word,c))
        # 同理也可以限制最多出现的词的数目
    dict_word2index = {word:c for word,c in enumerate(count)}
    dict_index2word ={c:word for word,c in enumerate(count)}
    print("vocab size:", len(count))
    print(count[-1])
    return count, dict_word2index, dict_index2word
count, dict_word2index, dict_index2word =bulid_vocabulary(data)


vocab size: 43727
('马福才', 4)


In [37]:
def bulid_dataset(ids,data,labels,dict_word2index,text_maxlen,num_classes):
    datasets=[]
    new_ids =[]
    new_labels =[]
    for i in range(len(labels)):
        new_ids.append('train_'+ids[i])
        multi_label = np.zeros(num_classes)
        for li in labels[i]:
            if(li>num_classes):
                print(li)
                labels[i].remove(li)
        multi_label[labels[i]] =1
        new_labels.append(multi_label)
        new_line =[]
        for word in data[i]:
            if word in dict_word2index:
                new_line.append(dict_word2index[word])
            else:
                new_line.append(0)#UNK
        pad_num = text_maxlen-len(new_line)
        while pad_num>0:
            new_line.append(1)
            pad_num-=1
        datasets.append(new_line[:text_maxlen])
    return new_ids,np.array(datasets,dtype=np.int64),np.array(new_labels, dtype=np.int64)

ids,data,multilabel = bulid_dataset(ids,data,law_labels,dict_word2index,text_maxlen=2000,num_classes=452)




In [23]:
import pickle
def save_dict(dic, fpath):
    output = open(fpath, 'wb')
    pickle.dump(dic, output)

def load_pickle(fpath):
    pkl_f = open(fpath, 'rb')
    return pickle.load(pkl_f)
save_dict(dict_word2index,'./word2idx.pkl')
save_dict(dict_index2word,'./idx2word.pkl')

In [21]:
def count_vocab_size(data):
    vocab = set()
    for text in data:
        vocab|=set(text)
    return len(vocab)
count_vocab_size(data)

132597

In [39]:
from torch.utils.data import Dataset
class MultiLawdata(Dataset):
    def __init__(self,ids,X,y):
        self.len = len(X)
        self.x_data = X
        self.ids = ids
        self.y_data = y
    def __getitem__(self, index):
        return self.ids[index],self.x_data[index],self.y_data[index]
    def __len__(self):
        return self.len
def split_data(data,ratio =0.7):
    index = int(len(data)*ratio)
    return data[:index],data[index:]
train_ids,valid_ids = split_data(ids)
train_data,valid_data = split_data(data)
train_labels,valid_labls = split_data(multilabel)
datasets =MultiLawdata(train_ids,train_data,train_labels)
 

In [40]:
del data


In [55]:
from torch.utils.data import DataLoader
train_loader = DataLoader(datasets,batch_size=20,shuffle=True,num_workers=0)


In [43]:
from torch import nn
class FastText(nn.Module):
    """
    fastText model
    """
    def __init__(self, config):
        super(FastText, self).__init__()
        self.is_training = True
        self.vocab_size = config.vocab_size
        self.embedding_size = config.embedding_size
        self.num_class = config.num_class
        
        self.embedding = nn.Embedding(num_embeddings=self.vocab_size, 
                                      embedding_dim=self.embedding_size)
        self.linear = nn.Linear(in_features=self.embedding_size, 
                                out_features=self.num_class)
   
    def forward(self, text):
        embed = self.embedding(text)
        text_embed = torch.mean(embed, dim=1)
#       
        logits = self.linear(text_embed)
        return logits    

    def get_optimizer(self, lr, lr2, weight_decay):

        return torch.optim.Adam([
            {'params': self.linear.parameters()},
            {'params': self.embedding.parameters(), 'lr': lr2}
        ], lr=lr, weight_decay=weight_decay)

In [49]:
class MultiConfig:
    

    batch_size = 32  # 64 or larger if has cuda
    step = 10000 // batch_size   # 3000 // batch_size if has cuda
    num_workers = 1
#    vocab_size = 241684
    vocab_size = 0
    #vocab_size = 0
    min_count = 5
    max_text_len = 2000
    embedding_size = 256
    num_class = 452
#    num_class = 321
    learning_rate = 0.001
    
    learning_rate2 = 0.0    # 0.0 if pre train emb
    lr_decay = 0.75
    begin_epoch = 2
    weight_decay = 0.0
    dropout_rate = 0.5
    epoch_num = 6
    epoch_step = max(1, epoch_num // 20)
   

   
  
config =MultiConfig()
config.vocab_size = len(dict_index2word)
model = FastText(config)
model = model.cuda()

In [57]:
loss_fun = nn.MultiLabelSdeoftMarginLoss()
optimizer = model.get_optimizer(config.learning_rate,
                                    config.learning_rate2,
                                    config.weight_decay)

In [48]:
len(dict_index2word)

43727

In [79]:
def get_multi_label_from_output(outputs):
    predicted = outputs.cpu().data.numpy()
    rows,cols = np.where(predicted>=0.2)
    predicted_multilabel = get_result(outputs.size()[0],rows,cols)
    predicted_maxlabel = torch.argmax(outputs,dim=1)
    predicted_len = len(predicted_maxlabel)
    for i in range(predicted_len):
        if predicted_multilabel[i]  == []:
            predicted_multilabel[i].append(predicted_maxlabel[i])
    return predicted_multilabel
            
def get_result(row_size,rows,cols):
    result =[]
    for i in range(row_size):
        result.append([])
        for ri,row in enumerate(rows):
            if row>i:
                break
            if row==i:
                result[row].append(cols[ri])
    return result 
def jaccard(predicted_label, true_label):
    # print("predicted labels: ", predicted_label)
    # print("true labels: ", true_label)
    p = 0
    N = len(predicted_label)
    predict_set_size = 0
    true_set_size = 0
    for i in range(N):
        Li = set(true_label[i])
        Lig = set(predicted_label[i])
        p += len(Li & Lig) / len(Li | Lig)
        
        true_set_size += len(Li)
        predict_set_size += len(Lig)

    print("predict_set_size / true_set_size: ", predict_set_size / true_set_size)
    return p / N
            

In [100]:
#this funtion is replace for nn.MultiLabelSdeoftMarginLoss()
def log_sigmoid(input):
    return torch.log(torch.sigmoid(torch.tensor(input)))
def loss_fun1(input,target):
    loss =-(target*log_sigmoid(input)+(1-target)*log_sigmoid(-input))
    loss = loss.mean(dim=1)
    return loss.sum()/len(input)
loss_fun1(model(inputs),labels.float())
loss_fun(model(inputs),labels.float())

C:\Users\Administrator\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


tensor(0.0252, device='cuda:0')

In [80]:
from torch.autograd import Variable
for epoch in range(3):
    print("lr:",config.learning_rate,"lr2:",config.learning_rate2)
    running_loss = 0.0
    for i,data in enumerate(train_loader):
        ids, text, label = data
        inputs,labels = Variable(text).cuda(),Variable(label).cuda()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fun(outputs,labels.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.data
        if i % 100 == 99:
           
            predict_label = get_multi_label_from_output(outputs)
            true_labels = labels.cpu().data.numpy()
            rows,cols = np.where(true_labels==1)
            true_labels =get_result(outputs.size()[0],rows,cols)
            
            running_jaccard =jaccard(predict_label,true_labels)
            
            print('[%d, %5d] loss: %.3f ,jaccard:%.3f' %
                    (epoch + 1, i + 1, running_loss / 100,running_jaccard))
            running_loss = 0.0
            
        
    

lr: 0.001 lr2: 0.0


predict_set_size / true_set_size:  0.25
[1,   100] loss: 0.024 ,jaccard:0.200


predict_set_size / true_set_size:  0.19801980198019803
[1,   200] loss: 0.024 ,jaccard:0.215


predict_set_size / true_set_size:  0.19827586206896552
[1,   300] loss: 0.025 ,jaccard:0.178


lr: 0.001 lr2: 0.0


...

predict_set_size / true_set_size:  0.2716049382716049
[2,   300] loss: 0.024 ,jaccard:0.183


lr: 0.001 lr2: 0.0


predict_set_size / true_set_size:  0.26666666666666666
[3,   100] loss: 0.025 ,jaccard:0.214


predict_set_size / true_set_size:  0.23958333333333334
[3,   200] loss: 0.025 ,jaccard:0.202


predict_set_size / true_set_size:  0.21649484536082475
[3,   300] loss: 0.024 ,jaccard:0.193


In [90]:
loss_fun(model(inputs),labels.float())

tensor(0.0252, device='cuda:0', grad_fn=<MeanBackward1>)

In [101]:
model(inputs).shape

torch.Size([20, 452])

In [98]:
torch.log(torch.sigmoid(torch.tensor(a)))

tensor([[-0.6601, -0.4950, -0.3710, -0.5720],
        [-0.4692, -0.4011, -0.5645, -0.3755],
        [-0.4310, -0.3171, -0.3196, -0.4620]], dtype=torch.float64)

In [97]:
a

array([[0.06719321, 0.44543959, 0.80040412, 0.25900788],
       [0.512974  , 0.7063108 , 0.2764153 , 0.78582022],
       [0.61831957, 0.98562428, 0.97660465, 0.53225032]])